# Karimi Proposal

In [1]:
#DO NOT CHANGE THIS CODE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, StratifiedShuffleSplit

import matplotlib.pyplot as plt
from auxiliar_funcs import *

import pmtools2 as pm
import kmodes
%matplotlib inline

In [2]:
# Load data
url_file = '../00-Data/cav_policies.csv'
cav_data = pd.read_csv(url_file)

# Get a smaller sample: 15K positive and 15k negative.
# cav_data = cav_data.groupby('result').sample(n=15000)
print("Columns: ", cav_data.columns)
print("Lenght: ", len(cav_data)); print()

user_attr = ['control', 'monitoring', 'fallback', 'weather', 'visibility', 
        'traffic_congestion']
#user_attr = ['control', 'monitoring', 'fallback']
rsrc_attr = ['driving_task_loa', 'vehicle_loa', 'region_loa']
cav_data = cav_data[user_attr + rsrc_attr + ['result']]

# Change string values to numerical
mapping = {'system': 10101, 'human': 10201, 'human and system': 10301} # Control
cav_data.control = cav_data.control.replace(mapping)

mapping = {'system': 20102, 'human': 20202} # monitoring
cav_data.monitoring = cav_data.monitoring.replace(mapping)

mapping = {'system': 30103, 'human': 30203} # fallbacj
cav_data.fallback = cav_data.fallback.replace(mapping)

mapping = {0: 40004, 1: 40104, 2: 40204, 3: 40304, 4: 40404, 5: 40504}
cav_data.driving_task_loa = cav_data.driving_task_loa.replace(mapping)

mapping = {0: 50005, 1: 50105, 2: 50205, 3: 50305, 4: 50405, 5: 50505}
cav_data.vehicle_loa = cav_data.vehicle_loa.replace(mapping)

mapping = {0: 60006, 1: 60106, 2: 60206, 3: 60306, 4: 60406, 5: 60506}
cav_data.region_loa = cav_data.region_loa.replace(mapping)


print("# User attr:", len(user_attr))
print("# Rsrc attr:", len(rsrc_attr)); print()

# Data statictics:
n_users = len(cav_data[user_attr].drop_duplicates())
n_rscrc = len(cav_data[rsrc_attr].drop_duplicates())
print("|U| =", n_users)
print("|R| =", n_rscrc); print()

# Add user and resource id columns
user_dict = get_user_res(cav_data, user_attr, True)
rsrc_dict = get_user_res(cav_data, rsrc_attr, False)
cav_data = add_col(cav_data, user_dict, user_attr, "USRID")
cav_data = add_col(cav_data, rsrc_dict, rsrc_attr, "RESID")

# Accepted and rejected requests
cav_pos = cav_data[cav_data.result == 'approved']
cav_neg = cav_data[cav_data.result == 'rejected']
print("|L+| =", len(cav_pos), "{:.2f}%".format((len(cav_pos) 
        / len(cav_data) ) * 100))
print("|L-| =", len(cav_neg), "{:.2f}%".format((len(cav_neg) 
        / len(cav_data) ) * 100))

# Cross validation
k = 10
test_size = 0.2
kfold = StratifiedShuffleSplit(n_splits=k, test_size=test_size, random_state=1)

data_partition = kfold.split(cav_data, cav_data.result)
data_curpus = [] # A list to storage the k folds

for train_data, test_data in data_partition:
    X_train, X_test = cav_data.iloc[train_data], cav_data.iloc[test_data]
    data_curpus.append([X_train, X_test])

print("Done!")  
print(" - k =", k)
print(" - Train-Test size: ", len(data_curpus[0][0]), "(", (1-test_size)*100, ") \t", len(data_curpus[0][1]), "(", test_size*100, ")")

Columns:  Index(['driving_task_type', 'control', 'monitoring', 'fallback', 'weather',
       'visibility', 'traffic_congestion', 'environmental_weighted_average',
       'driving_task_loa', 'vehicle_loa', 'region_loa', 'result'],
      dtype='object')
Lenght:  239580

# User attr: 6
# Rsrc attr: 3

|U| = 6655
|R| = 216

|L+| = 118975 49.66%
|L-| = 120605 50.34%
Done!
 - k = 10
 - Train-Test size:  191664 ( 80.0 ) 	 47916 ( 20.0 )


## Data pre-processing
1. Continouos to Categorical Values
2. Handle missing values - New value

In [3]:
id_kfold = 1

cav_train, cav_test = data_curpus[id_kfold][0], data_curpus[id_kfold][1]
print("# Train access request =", len(cav_train), "{:.2f}%".format(
    len(cav_train)/(len(cav_train)+len(cav_test))*100))
print("# Train access request =", len(cav_test), "{:.2f}%".format(
    len(cav_train)/(len(cav_train)+len(cav_test))*100))
print("Total =", len(cav_train)+len(cav_test)); print()

#### **** SELECT FUNCTIONAL ATTRIBUTES **** ####
cav_train = cav_train[user_attr + rsrc_attr + ['USRID', 'RESID', 'result']]
cav_test = cav_test[user_attr + rsrc_attr + ['USRID', 'RESID', 'result']]

##### ***** Task 1: Null and uknwokn values ***** #####
print("TASK 1: Done!"); print() # NA


##### ***** TASK 2: convert continuous values to categorical values ***** #####
print("TASK 2: Done!"); print() # NA 

##### ***** TASK 3: Drop duplicates access requests ***** #####
print("TASK 3: Drop duplicates access requests")

positive_cav_train = cav_train[cav_train.result=='approved']
positive_cav_test = cav_test[cav_test.result=='approved']
negative_cav_train = cav_train[cav_train.result=='rejected']
negative_cav_test = cav_test[cav_test.result=='rejected']


print(" -TRAIN DATA: Removing", 
    len(positive_cav_train.drop_duplicates()) - 
    len(positive_cav_train), "positive access requests")
print(" -TRAIN DATA: Removing", 
    len(negative_cav_train.drop_duplicates()) - 
    len(negative_cav_train), "negative access requests")
print(" -TEST DATA: Removing", 
    len(positive_cav_test.drop_duplicates()) - 
    len(positive_cav_test), "positive access requests")
print(" -TEST DATA: Removing", 
    len(negative_cav_test.drop_duplicates()) - 
    len(negative_cav_test), "negative access requests")

# Filter resources
#bolean_series = negative_cav_train.RESID.isin(top_list)
#negative_cav_train = negative_cav_train[bolean_series]
print("Hecho!")

# Train access request = 191664 80.00%
# Train access request = 47916 80.00%
Total = 239580

TASK 1: Done!

TASK 2: Done!

TASK 3: Drop duplicates access requests
 -TRAIN DATA: Removing 0 positive access requests
 -TRAIN DATA: Removing 0 negative access requests
 -TEST DATA: Removing 0 positive access requests
 -TEST DATA: Removing 0 negative access requests
Hecho!


## Selection of Learning Algorithm
1. K-modes algorithm

In [4]:
###Select the number of clusters###
num_clusters = 20

#DO NOT CHANGE THIS CODE
# seed = 29

#Compute centroids and labels
# num_init = 5
centroids = []
kmodes_huang = kmodes.KModes(n_clusters=num_clusters, init='Huang', verbose=0)
cluster_labels = kmodes_huang.fit_predict(positive_cav_train)
centroids = kmodes_huang.cluster_centroids_

print('Ready!')    

Ready!


In [5]:
positive_cav_train["cls"] = cluster_labels

/home/daniel/anaconda3/envs/tesis_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Parameter tuning
1. Number of clusters (Silhouette Method)
2. Cluster initialization

## Policy Rules Extraction

In [6]:
def freq(value, attribute, dataplace):
    """
    Calculate the frequency of the value in the dataplace.

    Parameters
    ----------
    value : int
        Value to compute its frequency.

    attribute : string
        Name of the attribute.

    dataplace : DataFrame pandas
        Data to search.
    
    Returns
    -------
    float [0-1]
        Returns the value of frequency of the value in the data.
    """
    value_freq = dataplace[dataplace[attribute] == value].drop_duplicates()
    return len(value_freq) / len(dataplace)

def freq_rels(attrA, attrB, dataplace):
    """
    Compute the frequency of the attribute relation.

    Parameters
    ----------
    attrA : string
        Name of the attribute A to compare.
    attrB : string
        Name of the attribute B to compare.
    dataplace : DataFrame pandas
        Data to search.

    Returns
    -------
    float [0-1]
        Returns the value of frequency of the range of values in the data.
    """
    # Get the range of values of attribute A.
    range_val_A = set(dataplace[attrA].values.tolist())

    # Get the range of values of attribute B.
    range_val_B = set(dataplace[attrB].values.tolist())

    # Check if the len
    if len(range_val_A) == len(range_val_B):
        # Compute the intersection
        inter_A_B = range_val_A.intersection(range_val_B)
        if len(inter_A_B) == len(range_val_A):
            boolean_series = dataplace[attrA].isin(inter_A_B)
            frac_log = dataplace[boolean_series]
            return len(frac_log) / len(dataplace) # Return the fraction
        return 0
    else:
        return 0    

def extract_attributes_filters(C_i, A, L, posThr, negThr):
    """
    Effective attribute extraction algorithm. Generate a rule for each cluster.

    Parameters
    ----------
    C_i : DataFrame pandas
        Access request in the Cluster i.

    A : List
        List of attributes.

    V : List
        Values of attributes.

    L : DataFrame
        Complete Access Log.

    PosThr : float
        Positive Threshold to the effective positive attribute.

    NegThr : float
        Negative Threshold to the effective negative attribute.

    Returns
    -------
    list
        Returns the rule with the effective attributes for the cluster i.
    """
    filter_to_ret = [] # Rule
    for a in A:        
        a_values = C_i[a].drop_duplicates().tolist()        
        for v in a_values:
            if freq(v, a, C_i) - freq(v, a, L) > posThr:
                if not [a, v] in filter_to_ret:
                    filter_to_ret.append([a, v])
            if freq(v, a, L) - freq(v, a, C_i) > negThr:
                if not [a, -1*v] in filter_to_ret:
                    filter_to_ret.append([a, v*-1])
    return filter_to_ret

def extract_relations(C_i, A, L, posThr, negThr):
    """
    Extract the effective relation. For each cluster.

    Parameters
    ----------
    C_i : DataFrame pandas
        Access request in the Cluster i.

    A : List
        List of attributes.

    L : DataFrame
        Complete Access Log.

    posThr : float
        Positive Threshold to the effective positive relation.

    negThr : float
        Negative Threshold to the effective negative relation.

    Returns
    -------
    list
        Returns the rule with the effective relation for the cluster i.
    """
    relation_to_ret = []
    for a in A:
        for b in A:
            if a != b:
                if freq_rels(a, b, C_i) - freq_rels(a, b, L) > posThr:
                    if not [a, b] in relation_to_ret:
                        relation_to_ret.append([a, b])
                if freq_rels(a, b, L) - freq_rels(a, b, C_i) > negThr:
                    if not [a, '!'+b] in relation_to_ret:                        
                        relation_to_ret.append([a, '!'+b])
                    print()

def rule_inference(data_, pos_attr_thr, 
    neg_attr_thr, pos_rel_thr, neg_rel_thr):
    rule_list = [] # All rules
    n_cluster = len(data_["cls"].drop_duplicates()) # N clusters
    attrs = data_.columns[:-1] # Name of the columns

    for C_i in range(n_cluster):
        #print(C_i)
        rule_i = []
        data_cluster = data_[data_["cls"] == C_i]
        
        # Effective attributes
        attr_filters = extract_attributes_filters(data_cluster, attrs, data_, 
            pos_attr_thr, neg_attr_thr)    
        rule_i.append(attr_filters)        

        # Relations
        attr_relation = extract_relations(data_cluster, attrs, data_, 
            pos_rel_thr, neg_rel_thr)
        rule_i.append(attr_relation)
        #print(rule_i)

        rule_list.append([C_i, rule_i])    

    return rule_list

In [7]:
df_test = positive_cav_train[['monitoring', 'fallback', 'weather', 'visibility', 'traffic_congestion',
       'driving_task_loa', 'vehicle_loa', 'region_loa', 'cls']]
df_test.columns

Index(['monitoring', 'fallback', 'weather', 'visibility', 'traffic_congestion',
       'driving_task_loa', 'vehicle_loa', 'region_loa', 'cls'],
      dtype='object')

In [8]:
pos_attr_thr = 0.3
neg_attr_thr = 0.3
pos_rel_thr = 0.2
neg_rel_thr = 0.2
test = rule_inference(df_test, pos_attr_thr, neg_attr_thr, pos_rel_thr, neg_attr_thr)
test

[[0,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['region_loa', 60106]],
   None]],
 [1, [[['driving_task_loa', 40004], ['vehicle_loa', 50105]], None]],
 [2,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['region_loa', 60406]],
   None]],
 [3,
  [[['driving_task_loa', 40004],
    ['vehicle_loa', 50005],
    ['region_loa', 60106]],
   None]],
 [4,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['region_loa', 60506]],
   None]],
 [5,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['vehicle_loa', 50205],
    ['region_loa', 60206]],
   None]],
 [6,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['vehicle_loa', 50405],
    ['region_loa', 60306]],
   None]],
 [7,
  [[['driving_task_loa', 40204],
    ['driving_task_loa', -40004],
    ['region_loa', 60206]],
   None]],
 [8,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['vehicle_loa', 50505],
    

In [9]:
only_rules = []
for rule in test:
    only_rules.append(rule[1][0])
only_rules

[[['driving_task_loa', 40104],
  ['driving_task_loa', -40004],
  ['region_loa', 60106]],
 [['driving_task_loa', 40004], ['vehicle_loa', 50105]],
 [['driving_task_loa', 40104],
  ['driving_task_loa', -40004],
  ['region_loa', 60406]],
 [['driving_task_loa', 40004], ['vehicle_loa', 50005], ['region_loa', 60106]],
 [['driving_task_loa', 40104],
  ['driving_task_loa', -40004],
  ['region_loa', 60506]],
 [['driving_task_loa', 40104],
  ['driving_task_loa', -40004],
  ['vehicle_loa', 50205],
  ['region_loa', 60206]],
 [['driving_task_loa', 40104],
  ['driving_task_loa', -40004],
  ['vehicle_loa', 50405],
  ['region_loa', 60306]],
 [['driving_task_loa', 40204],
  ['driving_task_loa', -40004],
  ['region_loa', 60206]],
 [['driving_task_loa', 40104],
  ['driving_task_loa', -40004],
  ['vehicle_loa', 50505],
  ['region_loa', 60206]],
 [['monitoring', 20102],
  ['monitoring', -20202],
  ['driving_task_loa', 40304],
  ['driving_task_loa', -40004]],
 [['driving_task_loa', 40004], ['vehicle_loa', 50

In [10]:
# Compute how many rules has a lenght equal to 1.
for idx, rule in enumerate(only_rules):
    if len(rule) == 1:
        print(idx)

In [11]:
false_neg  = []
for i,row in positive_cav_train.iterrows():
    
    # Evaluación
    denies_count = 0
    #print(row)
    
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True
        #print(rule)
        #for idx_r, attr_val in enumerate(rule):
        for attr_val in rule:            
            if attr_val[1] < 0:
                if row[attr_val[0]] == attr_val[1]*-1:
                    res = False
                    #print(res)
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1
    
    if denies_count == len(only_rules):
        false_neg.append(row)
        #print("FP-2")

In [12]:
FN = len(false_neg)
print("Tasa FN: {:.2f}".format((FN/ len(positive_cav_train))*100))
print("FN: ", FN, " de ", len(positive_cav_train))

Tasa FN: 45.47
FN:  43275  de  95180


In [13]:
false_pos  = []
for i,row in negative_cav_train.iterrows():
    # Evaluación
    denies_count = 0
    temp_rules_n = 0
    #print(row)
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True
        #print(rule)                     
        for attr_val in rule:
            if attr_val[1] < 0:
                #print(attr_val)
                if row[attr_val[0]] == attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1                                
    #print("XXX-", denies_count, temp_rules_n, res)
    #print("R", denies_count, len(only_rules))
    if denies_count < len(only_rules):
        false_pos.append(row)
        #print("FP-2")    
    #else:
        #print("ENtra PAPA")
    

In [14]:
FP = len(false_pos)
print("Tasa FP: {:.2f}".format((FP/ len(negative_cav_train))*100))
print("FN: ", FP, " de ", len(negative_cav_train))

Tasa FP: 37.38
FN:  36061  de  96484


In [15]:

TP = len(positive_cav_train) - FN
#TP = 50 - FN
TN = len(negative_cav_train) - FP
#TN = 50 - FP

precision = TP / (TP + FP)

recall = TP / (TP + FN)

fscore = 2*(precision*recall)/(precision+recall)

print("FN:", FN, " - {:.2f}".format((FN/len(positive_cav_train))*100))
#print("FN:", FN, " - {:.2f}".format((FN/50)*100))
print("FP:", FP, " - {:.2f}".format((FP/len(negative_cav_train))*100))
#print("FP:", FP, " - {:.2f}".format((FP/50)*100))
print("Precision:", precision)
print("Recall:", recall)
print("F-score", fscore)

def compute_wsc(policy):
    return sum([len(rule) for rule in policy])

print("# Rules:", len(only_rules))
print("WSC:", compute_wsc(only_rules))

FN: 43275  - 45.47
FP: 36061  - 37.38
Precision: 0.5900575222244958
Recall: 0.545335154444211
F-score 0.5668155460670722
# Rules: 20
WSC: 71


## Policy Enhancement
1. Rule Pruning
2. Policy Refinement

In [16]:
false_neg = pd.DataFrame(false_neg)

###Select the number of clusters###
num_clusters = 10

#DO NOT CHANGE THIS CODE
# seed = 29

#Compute centroids and labels
# num_init = 5
centroids = []
kmodes_huang = kmodes.KModes(n_clusters=num_clusters, init='Huang', verbose=0)
cluster_labels = kmodes_huang.fit_predict(false_neg)
centroids = kmodes_huang.cluster_centroids_

print('Ready!')    

Ready!


In [17]:
false_neg["cls"] = cluster_labels

In [18]:
df_test_2 = false_neg[['monitoring', 'fallback', 'weather', 'visibility', 
    'traffic_congestion', 'driving_task_loa', 'vehicle_loa', 'region_loa', 'cls']]

In [19]:
pos_attr_thr = 0.3
neg_attr_thr = 0.2
pos_rel_thr = 0.3
neg_rel_thr = 0.2
test = rule_inference(df_test_2, pos_attr_thr, neg_attr_thr, pos_rel_thr, neg_attr_thr)
test

[[0,
  [[['driving_task_loa', 40204],
    ['driving_task_loa', -40004],
    ['region_loa', 60506]],
   None]],
 [1,
  [[['driving_task_loa', 40204],
    ['driving_task_loa', -40004],
    ['region_loa', 60306]],
   None]],
 [2, [[['driving_task_loa', 40004], ['region_loa', -60306]], None]],
 [3,
  [[['driving_task_loa', 40104],
    ['driving_task_loa', -40004],
    ['region_loa', 60306]],
   None]],
 [4,
  [[['driving_task_loa', 40004],
    ['vehicle_loa', 50005],
    ['vehicle_loa', -50405],
    ['region_loa', -60506]],
   None]],
 [5, [[['driving_task_loa', 40004], ['vehicle_loa', 50505]], None]],
 [6, [[['driving_task_loa', 40004], ['vehicle_loa', 50405]], None]],
 [7,
  [[['driving_task_loa', 40004],
    ['vehicle_loa', 50405],
    ['vehicle_loa', -50505],
    ['region_loa', 60406],
    ['region_loa', -60306]],
   None]],
 [8,
  [[['visibility', 2], ['traffic_congestion', 0], ['driving_task_loa', 40004]],
   None]],
 [9,
  [[['driving_task_loa', 40004],
    ['region_loa', 60106],
  

In [20]:
#only_rules = []
for rule in test:
    only_rules.append(rule[1][0])
len(only_rules)

30

In [21]:
# Compute how many rules has a lenght equal to 1.
for idx, rule in enumerate(only_rules):
    if len(rule) == 1:
        print(idx)

In [25]:
false_neg  = []
for i,row in positive_cav_train.iterrows():
    
    # Evaluación
    denies_count = 0    
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True
        
        #for idx_r, attr_val in enumerate(rule):
        for attr_val in rule:            
            if attr_val[1] < 0:
                if row[attr_val[0]] == attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1
    
    if denies_count == len(only_rules):
        false_neg.append(row)
        #print("FP-2")

FN = len(false_neg)
print("Tasa FN: {:.2f}".format((FN/ len(positive_cav_train))*100))
print("FN: ", FN, " de ", len(positive_cav_train))

Tasa FN: 6.42
FN:  6110  de  95180


In [26]:
false_pos  = []
for i,row in negative_cav_train.iterrows():
    # Evaluación
    denies_count = 0
    temp_rules_n = 0
    for rule in only_rules:                                      
        # En esta parte se evalua la regla completa
        res = True                        
        for attr_val in rule:
            if attr_val[1] < 0:
                if row[attr_val[0]] == attr_val[1]*-1:
                    res = False
                    break
            else:
                if row[attr_val[0]] != attr_val[1]:
                    res = False
                    break
        if res == False:
            denies_count += 1                                
    #print("XXX-", denies_count, temp_rules_n, res)
    if denies_count < len(only_rules):
        false_pos.append(row)
        #print("FP-2")    
    #else:
    #    print("ENtra PAPA")
FP = len(false_pos)
print("Tasa FP: {:.2f}".format((FP/ len(negative_cav_train))*100))
print("FN: ", FP, " de ", len(negative_cav_train))

Tasa FP: 42.82
FN:  41313  de  96484


In [27]:
TP = len(positive_cav_train) - FN
#TP = 50 - FN
TN = len(negative_cav_train) - FP
#TN = 50 - FP

precision = TP / (TP + FP)

recall = TP / (TP + FN)

fscore = 2*(precision*recall)/(precision+recall)

print("FN:", FN, " - {:.2f}".format((FN/len(positive_cav_train))*100))
#print("FN:", FN, " - {:.2f}".format((FN/50)*100))
print("FP:", FP, " - {:.2f}".format((FP/len(negative_cav_train))*100))
#print("FP:", FP, " - {:.2f}".format((FP/50)*100))
print("Precision:", precision)
print("Recall:", recall)
print("F-score", fscore)

def compute_wsc(policy):
    return sum([len(rule) for rule in policy])

print("# Rules:", len(only_rules))
print("WSC:", compute_wsc(only_rules))

FN: 6110  - 6.42
FP: 41313  - 42.82
Precision: 0.6831412070592024
Recall: 0.9358058415633537
F-score 0.7897571853539809
# Rules: 30
WSC: 101
